In [30]:
import spacy
import random
import pandas as pd

from spacy.training.example import Example

In [31]:
# Load the dataset

df = pd.read_csv('../dataset/nou_hearings.csv')

# Drop the columns that are not needed
df = df["actor"].unique()
df = df[:10]

In [43]:
from spacy.tokens import doc
from thinc import loss


nlp = spacy.load("nb_core_news_md")

ner = nlp.get_pipe("ner")
ner.add_label("ORG2")
# Data for training

train_data = []
for i in range(len(df)):
    train_data.append((df[i], {"entities": [(0, len(df[i]), "ORG")]}))

# Train the NER model
optimizer = nlp.begin_training()

for batch in spacy.util.minibatch(train_data, size=5):
    for text, annotations in batch:
        doc = nlp.make_doc(text)
        example = Example.from_dict(doc, annotations)
        ner.update([example], drop=0.3, sgd=optimizer)
        #nlp.update([example], drop=0.3, sgd=optimizer)

nlp.to_disk("models/nb_core_news_md_more_orgs")
